In [1]:
from dataclasses import dataclass, field

import itertools
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize
from typing import Any, Callable, Mapping, Sequence
import xarray as xr

from HARK import distribution
from HARK.utilities import CRRAutility, CRRAutility_inv

In [2]:
from HARK.stage import Stage, backwards_induction

In [3]:
## Doing this because of the CRRAutility warnings

import warnings
warnings.filterwarnings('ignore')

# General Bellman Stage Form

This notebook demonstrates HARK's ability to represent and compose Bellman stages.
This is possible because all Bellman stages have a general form.

In each Bellman stage, the agent:
 - begins in some input states $\vec{X} \in \vec{X}$
 - experiences some exogeneous shocks $\vec{k} \in \vec{K}$
 - can choose some actions $\vec{a} \in \vec{A}$
 - subject to constraints $\Gamma: \vec{X} \times \vec{K} \times \vec{A} \rightarrow \mathbb{B}$
     - For scalar actions, these may be expressed as upper and lower bounds, such that $\Gamma_{lb} \lte a \lte \Gamma_{ub}$:
         - $\Gamma_{ub}: \vec{X} \times \vec{K} \rightarrow \mathbb{R}$
         - $\Gamma_{lb}: \vec{X} \times \vec{K} \vec{A} \rightarrow \mathbb{R}$
 - experience a reward $F: \vec{X} \times \vec{K} \times \vec{A} \rightarrow \mathbb{R}$
 - together, these determine some output states $\vec{y} \in \vec{Y}$ via...
 - a **deterministic** transition function $T: \vec{X} \times \vec{K} \times \vec{A} \rightarrow \vec{Y}$
   - _This is deterministic because shocks have been isolated to the beginning of the stage._
   - CDC thinks there needs to be an additional between-stage transition function.
 - The agent has a discount factor $\beta$ for future utility.
   - CDC: These can be a function of...

## Solving one stage

For any stage, consider two value functions.
 - $v_x : X \rightarrow \mathbb{R}$ is the value of its input states.
 - $v_y : Y \rightarrow \mathbb{R}$ is the value of its output states. Others migth write this $\mathfrak{v}$'
 
The stage is solved with respect to a value function $v_y : \vec{Y} \rightarrow \mathbb{R}$ over the output states. The $q: \vec{X} \times \vec{K} \times \vec{A} \rightarrow \mathbb{R}$ is the action-value of a state, shock, action combination.

$$q(\vec{x}, \vec{k}, \vec{a}) = F(\vec{x}, \vec{k}, \vec{a}) + \overbrace{\beta(\vec{x},\vec{k},\vec{a}) v_y(T(\vec{x}, \vec{k}, \vec{a}))}^{\hat{v}_y}$$

where $\beta$ is the agent's discount factor for that stage, and $\hat{v}_y : X \times K \times A \rightarrow \mathbb{R}$ is an alternative structure for the value funcion. Note that there is no expectation taking in this operation because $T$ is deterministic.

The optimal policy $\pi: \vec{X} \times \vec{K} \rightarrow \vec{A}$ is:

$$\pi^*(\vec{x}, \vec{k}) = \mathrm{argmax}_{\vec{a} \in \vec{A}} q(\vec{x}, \vec{k}, \vec{a})$$

(This is solved by griding over $x$ and $k$ ...)

The optimal policy $\pi^*$ can then be used to derive the value function over the input states $V_x: \vec{X} \rightarrow \mathbb{R}$.

$$v_x(\vec{x}) = \mathbb{E}_{\vec{k} \in \vec{K}}[q(\vec{x}, \vec{k}, \pi^*(\vec{x}, \vec{k}))]$$

Note that this requires no optimization, but does require the taking of expectations over the probability distribution over the shocks.

**TODO**: Include the transformed interpolation technique here. -- using the inverted utility function.

# Portfolio Choice Problem

We demonstrate the expressivity of the general Bellman stage form by representing a well-understood problem, the Portfolio Choice problem.

In this intertemporal choice problem, the agent has two decisions to make at each time step: $c$, how many resources to consume; and $\alpha$, what proportion of their savings to allocate to a risky asset. They earn income every time period; this income is subject to permanent ($\psi$) and transitory ($\theta$) shocks. The risky asset's growth is also subject to a shock ($\eta$). The agent is rewarded for consumption through a CRRA utility function governed by $\rho$.

This problem is implemented and analyzed in other HARK materials. In this notebook, we focus on representing the problem as a series of Bellman stages of the above form, implementing them using HARK's `stage` module, and solving the composed problem with a generic value iteration algorithm.

### Epsilon -- A Tiny Hack

We have a small technical problem which is that the CRRA utility function has a value of $-\infty$ when consumption is 0, and this throws off the optimizers.

As a hack, we will define $\epsilon$ (`epsilon` in Python) for the minimum value of resources/consumption that we will use in the discretization of the resource space.

In [4]:
epsilon = 1e-4

Optimizer args ...

In [9]:
float('-inf')

-inf

## Stages

First we define the stages. Then we combine the stages together and solve them recursively.

### Consumption stage

The consumption stage:

* $c \in A_0 = \mathbb{R}$
* $m \in X_0 = \mathbb{R}$
* $a \in Y_0 = \mathbb{R}$
* $\Gamma_0$ ... restricts consumption $c \leq m$
* $F_0(m,c) = CRRA(c, \rho)$
* $T_0(m,c) = m - c$ 
* $\beta_0 = \beta $

Requires a parameter $\rho$

In [10]:
CRRA = 5

consumption_stage = Stage(
    transition = lambda x, k, a : {'a' : x['m'] - a['c']}, 
    reward = lambda x, k, a : CRRAutility(a['c'], CRRA), 
    inputs = ['m'], 
    actions = ['c'],
    outputs = ['a'],
    action_upper_bound = lambda x, k: (x['m'],) , 
    action_lower_bound = lambda x, k: (0,) , # This is a kludge.
    #constraints = [lambda x, k, a: x['m'] - a['c'],
    #               lambda x, k, a: a['c'] - epsilon
    #              ], # has to be nonnegative to clear\n"
    discount = .96, # lambda x, k, a : .96 * k['psi']^(1 - CRRA) < --- 
    optimizer_args = {
        'method' : 'Nelder-Mead',
        'options' : {
            'maxiter': 50000,
        }
    },
    value_transform = np.exp,
    value_transform_inv = np.log,
    # For configuring the stage with given pi* values.
    pi_star_points = {((0.0,),()) : (0.0,)}
)

_Two points to add by hand to the consumption function_
 - (0, 0)
 - the kink point -- the point at which (or below) you will consume everything you have.
   - which can be solved for, leads to the EGM.
   - You can calculate the marginal value of ending the period at exactly 0.
   - Envelope theorem to calculate the level of consumption that yields the same marginal utility.

Given a value function over stage outputs, we can then compute components of the solution for this stage.

Here `pi_star` represents $\pi^*$, the optimal policy (action) at each input value in the given grid. (There are no shocks in this period, otherwise they would also be included as an argument of $\pi^*$.)

`q` represents $q$, the action-value given the input state $m$ and the optimal action $\pi^*(m)$.

In [11]:
def consumption_v_y(y : Mapping[str,Any]):
    return 0

pi_star, q = consumption_stage.optimal_policy(
    {'m' : [9, 11, 20, 300, 1000]},
    v_y = consumption_v_y
)

ZeroDivisionError: 0.0 cannot be raised to a negative power

In [12]:
q

NameError: name 'q' is not defined

Because we picked the trivial output value function $v_y = 0$, the best policy is to consume all available resources.

In [ ]:
pi_star

To compute a solution to the stage, the solver requires grids over the input and shock spaces as well as a output value function $v_y$.

In [ ]:
sol = consumption_stage.solve(
    {'m' : np.linspace(epsilon,50,25)}, {},
    v_y = consumption_v_y
)

The 'solution object' is an `xarray.Dataset` coordinated over the state and shock spaces that includes data for the $v_x$ input state value function, optimal policy $\pi^*$, and action-value function $q$.

In [ ]:
sol

### Allocation stage

The allocation stage. Note that this is a trivial transition function.:

* $\alpha \in A_1 = \mathbb{R}$
* $a \in X_1 = \mathbb{R}$
* $(a, \alpha) \in Y_1 = \mathbb{R}^2$
* $\Gamma_1$ ... restricts allocation $0 \leq \alpha \leq 1$
* $F_1(a,\alpha) = 0$
* $T_1(a,\alpha) = (a, \alpha)$
* $\beta_1 = 1 $

In [ ]:
allocation_stage = Stage(
    transition = lambda x, k, a : {'a' : x['a'], 'alpha' : a['alpha']}, 
    inputs = ['a'], 
    actions = ['alpha'],
    outputs = ['a', 'alpha'],
    # Using bounds instead of constraints will result in a different optimizer
    action_upper_bound = lambda x, k: (1,) , 
    action_lower_bound = lambda x, k: (0,) ,
    value_transform = np.exp,
    value_transform_inv = np.log
)

Optimize portfolio allocation $\alpha$ with a more complex value function:

In [ ]:
def allocation_v_y(y : Mapping[str,Any]):
    return 0

pi_star, q = allocation_stage.optimal_policy(
    {'a' : [9, 11, 20, 300, 4000, 5500]},
    v_y = allocation_v_y
)

q

In [ ]:
pi_star

### Growth stage

The growth stage stage:

* $A_2 = \emptyset$
* $(a, \alpha) \in X_2 = \mathbb{R}^2$
* $m \in Y_0 = \mathbb{R}$
* Shocks:
    * $\psi \sim \text{Lognormal}(0,\sigma_\psi)$
    * $\theta \sim \text{Lognormal}(0,\sigma_\theta)$
    * $\eta \sim \text{Lognormal}(0,\sigma_\eta)$
* $F_2(a,\alpha) = 0$
* $T_2(a,\alpha) =  \frac{(\alpha \eta + (1 - \alpha) R) a + \theta}{\psi G} $ 
* $\text{discount}(\psi) = \beta L \psi^{1-\rho}$ ?? where $L$ is chance to remain alive

Requires parameters $R$ and $G$

In [ ]:
R = 1.01
G = 1.02

sigma_psi = 1.05
sigma_theta = 1.15
sigma_eta = 1.1
p_live = 0.98

def growth_transition(x, k, a): 
    return {'m' : ((x['alpha'] * k['eta'] + (1 - x['alpha']) * R) 
                   * x['a'] + k['theta']) 
            / (k['psi'] * G)}

growth_stage = Stage(
    transition = growth_transition,
    inputs = ['a', 'alpha'],
    discount = lambda x, k, a: p_live * k['psi'] ** (1 - CRRA), 
    shocks = {
        'psi' : distribution.Lognormal(0, sigma_psi),
        'theta' : distribution.Lognormal(0, sigma_theta),
        'eta' : distribution.Lognormal(0, sigma_eta),
        # 'live' : distribution.Bernoulli(p_live) ## Not implemented for now
    },
    outputs = ['m'],
)

In [ ]:
def growth_v_y(y : Mapping[str,Any]):
    return CRRAutility(y['m'], CRRA) # * 'live' ?

pi_star, q = growth_stage.optimal_policy(
    {'a' : [300, 600],
     'alpha' : [0, 1.0]
    },
    {'psi' : [1., 1.1], 
     'theta' : [1., 1.1], 
     'eta' : [1., 1.1],
     # 'live' : [0, 1] 
    }, 
    v_y = growth_v_y)

q

Because there are no actions to optimize, the $\pi^*$ function data is 'not a number'.

In [ ]:
pi_star

In [ ]:
sol = growth_stage.solve(
    {'a' : [0, 500, 1000], 'alpha' : [0, 0.5, 1.0]},
    {
        'psi' : 4, 
        'theta' : 4, 
        'eta' : 4,
     # 'live' : [0, 1] 
    }, growth_v_y)

sol.dataset

Note that solving this stage involves computing over a large grid, as it is the product of the sizes of the input and shock spaces.

In [ ]:
sol.dataset.map(np.exp)['v_x'].interp(a = 10, kwargs={"fill_value": 'extrapolate'}).to_dataset().map(np.log)['v_x']

In [ ]:
sol = growth_stage.solve(
    {'a' : [epsilon, 250, 500, 750, 1000], 'alpha' : [epsilon, 0.2, 0.4, 0.6, 0.8, 1.0]},
    {
        'psi' : 4, 
        'theta' : 4, 
        'eta' : 4,
     # 'live' : [0, 1] 
    }, growth_v_y)

sol.v_x({'a' : 300, 'alpha' : 0.25})

In [ ]:
sol.dataset.data_vars

## Solving in sequence

In [ ]:
x_grid = np.linspace(0,25,35)
alpha_grid = np.linspace(0,1,15)

shock_approx_params = {
            'psi' : 5, 
            'theta' : 5, 
            'eta' : 5,
        }

Solve the growth stage with the terminal utility of $0$.

In [ ]:
def growth_v_y_terminated(y : Mapping[str,Any]):
    return 0

g_sol = growth_stage.solve(
    {'a' : x_grid, 'alpha' : alpha_grid},
    shock_approx_params,
    v_y = growth_v_y_terminated
)

Solve allocation stage with value function from previous step.

In [ ]:
a_sol = allocation_stage.solve(
    {'a' : x_grid},
    {},
    v_y = g_sol.v_x
)

Solve the consumption stage with the value function from the previous step.

In [ ]:
c_sol = consumption_stage.solve(
    {'m' : x_grid},
    {},
    v_y = a_sol.v_x
)

The consumption function looks right! Consume all the resources!

In [ ]:
c_sol.dataset.data_vars['pi*'].plot(x='m')

The value function data does not support a good linear interpolation because the true function is so curved.

In [ ]:
c_sol.dataset.data_vars['v_x'].plot(marker="o",ls=":",x='m')

In [ ]:
m = np.linspace(-3,25,500)
plt.plot(m, c_sol.v_x({'m' : m}), label = 'v_x (with transform)')
#c_sol.dataset.map(np.exp).data_vars['v_x'].plot(marker="o",ls=":",x='m')
#plt.plot(m, sols[0].v_x({'m' :m}), label = 'v_x')
plt.legend()

**TODO**: Do the interpolation in the decurved space. (inverse utility)

Now, let's do the next round.

In [ ]:
g_sol = growth_stage.solve(
    {'a' : x_grid, 'alpha' : alpha_grid},
    shock_approx_params,
    v_y = c_sol.v_x
)

In [ ]:
g_sol.dataset.map(np.exp).data_vars['v_x'].plot(x=None)

In [ ]:
m = np.linspace(-3,25,500)
plt.plot(m, g_sol.v_x({'a' : m, 'alpha' : 0.5}), label = 'v_x (with transform)')
plt.legend()

In [ ]:
a_sol = allocation_stage.solve(
    {'a' : x_grid},
    {},
    v_y = g_sol.v_x
)

In [ ]:
a_sol

In [ ]:
a_sol.dataset.map(np.exp).data_vars['v_x'].plot(x=None)

In [ ]:
m = np.linspace(-3,25,500)
plt.plot(m, a_sol.v_x({'a' : m}), label = 'v_x (with transform)')
#c_sol.dataset.map(np.exp).data_vars['v_x'].plot(marker="o",ls=":",x='m')
#plt.plot(m, sols[0].v_x({'m' :m}), label = 'v_x')
plt.legend()

In [ ]:
a_sol.dataset.data_vars['pi*'].plot(x=None)

In [ ]:
c_sol = consumption_stage.solve(
    {'m' : x_grid},
    {},
    v_y = a_sol.v_x
)

In [ ]:
c_sol.dataset.data_vars['v_x'].plot(x='m')

In [ ]:
c_sol.dataset.data_vars['pi*'].plot(x='m')

## Curving the value function



In [ ]:
a_sol.dataset.data_vars['v_x']

## Backward Induction Solver

We chain together the stages and iterative solve each stage backwards, feeding $v_x$ into $v_y$, until convergence.

We will use a very coarse grid to start with, for demonstration purposes.

In [ ]:
stages_data = [
    #{
    #    'stage' : consumption_stage,
    #   'x_grid' : {'m' : np.linspace(epsilon*2,50,25)},
    #    'optimizer_args' :{
    #        'a0f' : lambda x: x['m'] - epsilon
    #    }
    #},
    {
        'stage' : allocation_stage,
        'x_grid' : {'a' : np.linspace(epsilon*2,50,25)},
        'optimizer_args' :{}
    },
    {
        'stage' : growth_stage,
        'x_grid' : {
            'a' : np.linspace(epsilon*2,50,25),
            'alpha' : np.linspace(0,1,11)
        },
        'shock_approx_params' : {
            'psi' : 3, 
            'theta' : 3, 
            'eta' : 3,
        },
    }
]

We can then solve each stage with backwards induction. Let's do this for one time-period, starting with the trivial terminal value function $v_y = 0$.

In [ ]:
def growth_v_y_terminal(y : Mapping[str,Any]):
    return 0

In [ ]:
sols = backwards_induction(stages_data, growth_v_y_terminal)

In [ ]:
sols[0].v_x({'a' : 3})

In [ ]:
sol = consumption_stage.solve(
    {'m' : np.linspace(epsilon*2,50,25)}, {},
    v_y = sols[1].v_x
)

In [ ]:
sol.dataset.data_vars['v_x']

In [ ]:
sols[1].dataset.data_vars['v_x'].plot(x='a')

In [ ]:
sols[1].v_x({'a' : 8.333499999999999 - 8.12633439})

In [ ]:
sols[0].v_x({'m' : np.linspace(epsilon,50,25)})

We can then retrieve the solution object for the first stage, a consumption stage.

In [ ]:
sols[0]

We find that as expected, the optimal policy $\pi^*$ of the consumption stage is to consume all available resources.

In [ ]:
sols[0].dataset.data_vars['pi*'].plot(x='m')

Next, we can take the _input_ value function $v_x$ of this consumption stage, and use it as the final _output_ function for another round of backwards induction.

First, let's look at $v_x$. It is a close approximation of the reward function for the consumption stage.
This is expected!

In [ ]:
f = lambda c : consumption_stage.reward({},{}, {'c':c})
m = np.linspace(1,25,200)
plt.plot(m, f(m), label = 'R')
plt.plot(m, sols[0].v_x({'m' :m}), label = 'v_x')
plt.legend()
#sols[0].dataset.data_vars['v_x'].plot(x='m', ax = ax)

Now we can run backwards induction with the new (penultimate) value function:

In [ ]:
sols = backwards_induction(stages_data, sols[0].v_x)

Now we have a new consumption function!

It is not smooth because of the coarseness of the discretization grid.

In [ ]:
sols[0].dataset.data_vars['pi*'].plot(x='m')

#### Alternative Composition: splitting independent shocks

Above, we found that the growth stage had a very large grid because of the size of the shock space. One optimization we can do with generalized Bellman stages is break down the effects of each independent shock into a separate stage.


#### $\eta$ Risky growth

The growth stage stage:

* $A_2 = \emptyset$
* $(a, \alpha) \in X_2 = \mathbb{R}^2$
* $a_\eta \in Y_0 = \mathbb{R}$
* Shocks:
    * $\eta \sim \text{Lognormal}(0,\sigma_\eta)$
* $F_2(a,\alpha) = 0$
* $T_2(a,\alpha) =  \alpha \eta + (1 - \alpha) R$
* $\beta = L$

Requires parameter $R$.

#### $\theta$ Transitory Growth

* $A_2 = \emptyset$
* $a_\eta \in X_2 = \mathbb{R}$
* $a_\theta \in Y_0 = \mathbb{R}$
* Shocks:
    * $\theta \sim \text{Lognormal}(0,\sigma_\theta)$
* $F_2(a,\alpha) = 0$
* $T_2(a,\alpha) =  a_\eta + \theta$ 

#### $\psi$ Permanent growth

* $A_2 = \emptyset$
* $a_\theta \in X_2 = \mathbb{R}$
* $m \in Y_0 = \mathbb{R}$
* Shocks:
    * $\psi \sim \text{Lognormal}(0,\sigma_\psi)$
* $F_2(a,\alpha) = 0$
* $T_2(a,\alpha) =  \frac{a_\theta}{R\psi}$
* $\beta(\psi) = \psi^{1 - \rho}$ ??

In [ ]:
def risky_growth_transition(x, k, a): 
    return {'a_eta' : ((x['alpha'] * k['eta'] + (1 - x['alpha']) * R))}

risky_growth_stage = Stage(
    transition = growth_transition,
    inputs = ['a', 'alpha'],
    discount = lambda x, k, a: p_live, 
    shocks = {
        'eta' : distribution.Lognormal(0, sigma_eta),
        # 'live' : distribution.Bernoulli(p_live) ## Not implemented for now
    },
    outputs = ['a_eta'],
)

transitory_growth_stage = Stage(
    transition = lambda x, k, a : {'a_theta' : (x['a_eta'] + k['theta'])},
    inputs = ['a_eta'],
    shocks = {
        'theta' : distribution.Lognormal(0, sigma_theta),
        # 'live' : distribution.Bernoulli(p_live) ## Not implemented for now
    },
    outputs = ['a_theta'],
)

permanent_growth_stage = Stage(
    transition = lambda x, k, a : {'a_theta' : (x['a_theta'] / ( k['psi'] * G))},
    inputs = ['a_theta'],
    discount = lambda x, k, a: k['psi'] ** (1 - CRRA), 
    shocks = {
        'psi' : distribution.Lognormal(0, sigma_psi),
        # 'live' : distribution.Bernoulli(p_live) ## Not implemented for now
    },
    outputs = ['m'],
)